<a href="https://colab.research.google.com/github/InhyeokYoo/Pytorch-study/blob/master/Ch%206.%20RNN/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6장

## BackPropagation Through Time (BPTT)

RNN의 기본수식과 계산은 다음과 같다:    
$$
\begin{aligned}
s_t = \tanh(Ux_t + Ws_{t-1}) \\
\hat{y}_t = softmax(Vs_t)
\end{aligned}
$$    
Loss는 Cross entropy를 사용하였고, 그 식은 아래와 같다:    

$$
\begin{aligned}
E(y_t, \hat{y_t})=-y_t \log \hat{y_t} \\
E(y, \hat{y})=-\sum_tE(y_t, \hat{y_t}) \\
= -\sum_t-y \log \hat{y_t}
\end{aligned}
$$  
여기서 총 loss는 매 시간마다의 loss의 총 합으로 나타낸다.  
목표는 파라미터 U, V, W에 대한 에러의 gradient를 계산해서 SGD를 이용해 좋은 파라미터를 찾는 것이다. 에러들을 모두 더하듯, 매 시간 스텝에서의 gradient도 하나의 학습 데이터에 대해 모두 더해준다:
$$
\begin{aligned}
\frac{\partial{E}}{\partial{W}}=\sum_t{\frac{\partial{E_t}}{\partial{W}}}
\end{aligned}
$$
![](http://www.wildml.com/wp-content/uploads/2015/10/rnn-bptt1.png)

$$
\begin{aligned}
\frac{\partial{E_3}}{\partial{V}}=\frac{\partial{E_3}}{\partial{\hat{y_3}}}\frac{\partial{\hat{y_3}}}{\partial{V}} \\
  =\frac{\partial{E_3}}{\partial{\hat{y_3}}}\frac{\partial{\hat{y_3}}}{\partial{z_3}}\frac{\partial{\hat{z_3}}}{\partial{V}} \\
 = (\hat{y_3}-y_3)\times s_3
\end{aligned}
$$
위 식에서 $z_3=Vs_3$이고, $\times $는 두 벡터의 외적이다. 핵심 포인트는 $\frac{\partial{E_3}}{\partial{V}}$가 현재 시간 스텝의 $\hat{y_3}, y_3, s_3$에만 의존한다는 점이다. 이 세 값을 갖고 있다면, V에 대한 gradient를 구하는 것은 단순 행렬연산이 된다.  

그러나 $\frac{\partial{E_3}}{\partial{W}}$에 대해서는 (U도 마찬가지) 상황이 조금 다르다. 이를 살펴보기 위해 위와 같이 chaine rule을 하면,
$$
\begin{aligned}
\frac{\partial{E_3}}{\partial{W}}=\frac{\partial{E_3}}{\partial{\hat{y_3}}}\frac{\partial{\hat{y_3}}}{\partial{s_3}}\frac{\partial{s_3}}{\partial{W}}
\end{aligned}
$$  
여기서 $s_3=\tanh(Ux_3+Ws_2)$는 $s_2$에 의존하고, $s_2$는 W와 $s_1$에 의존해서 chain rule이 이어진다. 따라서, W에 대한 미분을 하기 위해서는 $s_2$를 단순 상수 취급해서는 안된다. 다시 chain rule을 적용하면:
$$
\begin{aligned}
\frac{\partial{E_3}}{\partial{W}}=\sum_{k=0}^{3}\frac{\partial{E_3}}{\partial{\hat{y_3}}}\frac{\partial{\hat{y_3}}}{\partial{s_3}}\frac{\partial{s_3}}{\partial{s_k}}\frac{\partial{s_k}}{\partial{W}}
\end{aligned}
$$  
각 시간 스텝이 gradient에 기여하는 것을 전부 더해준다. 즉, W는 우리가 현재 처리 중인 출력 부분까지의 모든 시간 스텝에서 사용되기 때문에, $t=3$부터 $t=0$까지 gradient를 전부 backpropagation해 주어야 한다.
![](http://www.wildml.com/wp-content/uploads/2015/10/rnn-bptt-with-gradients.png) 

# 6.3 모델 구현, 학습 및 결과 확인

In [0]:
# 단순한 문자 RNN을 만들어보겠습니다.

import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [0]:
# 하이퍼파라미터 설정

n_hidden = 35 
lr = 0.01
epochs = 1000

In [0]:
string = "hello pytorch. how long can a rnn cell remember? show me your limit!"
chars =  "abcdefghijklmnopqrstuvwxyz ?!.,:;01"

char_list = [i for i in chars]
n_letters = len(char_list)

In [0]:
# 문자를 그대로 쓰지않고 one-hot 벡터로 바꿔서 연산에 쓰도록 하겠습니다.

#Start = [0 0 0 … 1 0]
#a =     [1 0 0 … 0 0]
#b =     [0 1 0 … 0 0]
#c =     [0 0 1 … 0 0]
#...
#end =   [0 0 0 … 0 1]

In [0]:
# 문자열을 one-hot 벡터의 스택으로 만드는 함수
# abc -> [[1 0 0 … 0 0],
#         [0 1 0 … 0 0],
#         [0 0 1 … 0 0]]

def string_to_onehot(string):
    start = np.zeros(shape=n_letters, dtype=int) # 0
    end = np.zeros(shape=n_letters, dtype=int) # 1
    start[-2] = 1
    end[-1] = 1

    for i in string:
        # 문자의 index를 찾음.
        idx = char_list.index(i)
        # 0으로만 구성된 배열을 만들어줌
        zero = np.zeros(shape=n_letters, dtype=int)
        # 해당 인덱스를 1로 만듬
        zero[idx] = 1
        # start와 새로 생긴 zero를 붙이고, 이를 start에 할당함.
        # np.vstack(tup) -> 열이 같은 두 개 이상의 배열을 vertical로 합침.
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

for i in zip(string_to_onehot(string)[1:-1], string):
    print(i)

(array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'h')
(array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'e')
(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'l')
(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'l')
(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'o')
(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]), ' ')
(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'p')
(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 

In [0]:
# 원-핫 벡터를 다시 문자로 바꾸는 함수:

def onehot_to_word(onehot_1):
    # 텐서를 입력으로 받아 넘파이 배열로 바꿔줍니다.
    onehot = torch.Tensor.numpy(onehot_1)
    # one-hot 벡터의 최대값(=1) 위치 인덱스로 문자를 찾습니다.
    return char_list[onehot.argmax()]

one_hot = string_to_onehot(string)
''.join([onehot_to_word(torch.from_numpy(i)) for i in one_hot])

'0hello pytorch. how long can a rnn cell remember? show me your limit!1'

In [0]:
# RNN with 1 hidden layer

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size) # h_t
        self.i2o = nn.Linear(input_size + hidden_size, output_size) # y_t
        self.act_fn = nn.Tanh()

    def forward(self, input, hidden):
        # 입력과 hidden state를 concat함.
        concated = torch.cat((input, hidden), 1)
        # Q. 굳이 이렇게 하는 이유를 잘 모르겠음.
        # s_t = \tanh(Ux_t + Ws_{t-1}), \hat{y}_t = softmax(Vs_t) 이므로, U(W_ih)랑 W(W_hh)만 update하면 되는 거 아닌가?

        # 이를 i2h 및 i2o에 통과시켜 hidden state는 업데이트, 결과값은 계산해줌.
        hidden = self.act_fn(self.i2h(concated))
        output = self.i2o(concated)

        return output, hidden
    
    # t=0일 때 초기화
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

rnn = RNN(n_letters, n_hidden, n_letters)

In [0]:
# Loss와 Optimzer

loss_fnc = nn.MSELoss() 
# Q. MSE 쓰는게 이상하지 않나.

optimizer = optim.Adam(rnn.parameters(), lr=lr)

In [0]:
### Training

# 문자열을 one-hot vector로 만들고, 이를 tensor로 바꾸어 줌.
# 또한, 데이터 타입도 학습에 맞게 바꾸어준다.
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor()) # 굳이 float tensor여야 하는 이유가...?

for i in range(epochs):
    optimizer.zero_grad()
    # h_0 초기화
    hidden = rnn.init_hidden() #h(t=0)

    # 문자열 전체에 대한 loss
    total_loss = 0
    for j in range(one_hot.size()[0]-1): # 마지막 input에 대해서는 output밖에 없음.
        # input: t 시점의 글자
        input_ = one_hot[j:j+1, :] # 1x35 -> cat함수 적용하기 위해서.
        # output: t+1 시점의 글자
        target = one_hot[j+1] # 35짜리 vector
        output, hidden = rnn.forward(input_, hidden) # 매 t에서 hidden을 뽑아내고, t+1에서 hidden을 다시 input으로 집어넣는다.
        
        loss = loss_fnc(output.view(-1), target.view(-1)) # 1 x 35 로 바꿔줌.
        total_loss += loss

    total_loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(loss)

IndexError: ignored

In [0]:
### Inference

start = torch.zeros(1,n_letters)
start[:,-2] = 1

rnn.eval()

hidden = rnn.init_hidden()
input_ = start # [BOS] token
output_string = ""

for i in range(len(string)):
    output, hidden = rnn.forward(input_, hidden)
    # Q. hidden은 share되는 parameter인데 명시적으로 주는 이유가?

    # 결과값을 문자로 바꿔서 output_string에 붙여줍니다.
    output_string += onehot_to_word(output.data)
    # 또한 이번의 결과값이 다음의 입력값이 됩니다.
    input_ = output

print(output_string)

hello pytorch. how eonn ceml r yr r mbmr alin eemlir?yeuc l nn ee e 


# Experiment

## Concat 안 썼을 때 실험결과는 어떨까?

In [0]:
# RNN with 1 hidden layer

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # input -> hidden
        self.i2h = nn.Linear(input_size, hidden_size)
        # reccurent
        self.h2h = nn.Linear(hidden_size, hidden_size)
        # output
        self.i2o = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()

    def forward(self, input_, hidden):
        # concat 없이 진행해보자.

        # 이를 i2h 및 i2o에 통과시켜 hidden state는 업데이트, 결과값은 계산해줌.
        hidden = self.act_fn(self.i2h(input_) + self.h2h(hidden))
        output = self.i2o(hidden)

        return output, hidden
    
    # t=0일 때 초기화
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

rnn = RNN(n_letters, n_hidden, n_letters)

In [0]:
### Training:
# Loss와 Optimzer

loss_fnc = nn.MSELoss() 
# Q. MSE 쓰는게 이상하지 않나.

optimizer = optim.Adam(rnn.parameters(), lr=lr)

### Training

# 문자열을 one-hot vector로 만들고, 이를 tensor로 바꾸어 줌.
# 또한, 데이터 타입도 학습에 맞게 바꾸어준다.
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor()) # 굳이 float tensor여야 하는 이유가...?

rnn.train()
for i in range(epochs):
    optimizer.zero_grad()
    # h_0 초기화
    hidden = rnn.init_hidden() #h(t=0)

    # 문자열 전체에 대한 loss
    total_loss = 0
    for j in range(one_hot.size()[0]-1): # 마지막 input에 대해서는 output밖에 없음.
        # input: t 시점의 글자
        input_ = one_hot[j] # 35짜리 vector이면 되니까 변경안해도 됨.
        # output: t+1 시점의 글자
        target = one_hot[j+1] # 35짜리 vector
        output, hidden = rnn(input_, hidden) # 매 t에서 hidden을 뽑아내고, t+1에서 hidden을 다시 input으로 집어넣는다.
        
        loss = loss_fnc(output.view(-1), target.view(-1)) # 1 x 35 로 바꿔줌.
        total_loss += loss

    total_loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(loss)

tensor(0.0530, grad_fn=<MseLossBackward>)
tensor(0.0174, grad_fn=<MseLossBackward>)
tensor(0.0119, grad_fn=<MseLossBackward>)
tensor(0.0068, grad_fn=<MseLossBackward>)
tensor(0.0032, grad_fn=<MseLossBackward>)
tensor(0.0016, grad_fn=<MseLossBackward>)
tensor(0.0010, grad_fn=<MseLossBackward>)
tensor(0.0007, grad_fn=<MseLossBackward>)
tensor(0.0009, grad_fn=<MseLossBackward>)
tensor(0.0007, grad_fn=<MseLossBackward>)
tensor(0.0009, grad_fn=<MseLossBackward>)
tensor(0.0003, grad_fn=<MseLossBackward>)
tensor(0.0003, grad_fn=<MseLossBackward>)
tensor(0.0002, grad_fn=<MseLossBackward>)
tensor(0.0003, grad_fn=<MseLossBackward>)
tensor(0.0002, grad_fn=<MseLossBackward>)
tensor(0.0002, grad_fn=<MseLossBackward>)
tensor(0.0001, grad_fn=<MseLossBackward>)
tensor(0.0002, grad_fn=<MseLossBackward>)
tensor(0.0001, grad_fn=<MseLossBackward>)
tensor(0.0007, grad_fn=<MseLossBackward>)
tensor(0.0002, grad_fn=<MseLossBackward>)
tensor(0.0001, grad_fn=<MseLossBackward>)
tensor(0.0002, grad_fn=<MseLossBac

In [0]:
### Inference

start = torch.zeros(1,n_letters)
start[:,-2] = 1

rnn.eval()

hidden = rnn.init_hidden()
input_ = start # [BOS] token
output_string = ""

for i in range(len(string)):
    output, hidden = rnn.forward(input_, hidden)
    # Q. hidden은 share되는 parameter인데 명시적으로 주는 이유가?

    # 결과값을 문자로 바꿔서 output_string에 붙여줍니다.
    output_string += onehot_to_word(output.data)
    # 또한 이번의 결과값이 다음의 입력값이 됩니다.
    input_ = output

print(output_string)

hell lomr llo?i llomilllo illlom lolor llo?i loo?i llooi lloom lloom


Concat한게 확실히 결과가 더 낫긴하네. 아무래도 문맥정보가 주어져서 인듯.

## RNN module 이용해보기


In [0]:
# input_size – 입력의 feature dim
# hidden_size – hidden state h의 dim
# num_layers – RNN layer의 수. 예를 들어 2일 경우 stacked 된 RNN 모듈 두 개를 사용하고, 두번 째 RNN은 첫번째 RNN에서 입력 받아 output을 냄. Default: 1
# nonlinearity – The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'
# bias – Default: True
# batch_first – True면, ouput 및 hidden state가 (batch, seq, feature)로 구성됨. Default: False
# dropout – If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
# bidirectional – If True, becomes a bidirectional RNN. Default: False

rnn_module = torch.nn.RNN(input_size=3, hidden_size=5)

# input: (seq_len, batch, input_size) 사이즈를 갖음.
input_ = 

# hidden: [num_layers * num_directions, batch, hidden_size]

## MSELoss 말고 다른거 써보기

Softmax - NLL 사용하기 위해
- output: [1 x 35]. softmax를 적용하여 35차원에 걸쳐 나눠져 있음.
- targt:  [1]. batch_size만큼 차원이 있고, class indices로 주어짐.

실험 결과 굉장히 안 좋음.

In [0]:
# RNN with 1 hidden layer

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size) # h_t
        self.i2o = nn.Linear(input_size + hidden_size, output_size) # y_t
        self.act_fn = nn.Tanh()

    def forward(self, input, hidden):
        # 입력과 hidden state를 concat함.
        concated = torch.cat((input, hidden), 1)
        # Q. 굳이 이렇게 하는 이유를 잘 모르겠음.
        # s_t = \tanh(Ux_t + Ws_{t-1}), \hat{y}_t = softmax(Vs_t) 이므로, U(W_ih)랑 W(W_hh)만 update하면 되는 거 아닌가?

        # 이를 i2h 및 i2o에 통과시켜 hidden state는 업데이트, 결과값은 계산해줌.
        hidden = self.act_fn(self.i2h(concated))
        output = self.i2o(concated)
        # output = nn.functional.softmax(output, dim=-1)
        output = nn.functional.log_softmax(output, dim=-1)
        return output, hidden
    
    # t=0일 때 초기화
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

rnn = RNN(n_letters, n_hidden, n_letters)

In [0]:
# Loss와 Optimzer

loss_fnc = nn.NLLLoss() # Softmax 써보자.

optimizer = optim.Adam(rnn.parameters(), lr=lr)

### Training

# 문자열을 one-hot vector로 만들고, 이를 tensor로 바꾸어 줌.
# 또한, 데이터 타입도 학습에 맞게 바꾸어준다.
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor()) # Q. 굳이 float tensor여야 하는 이유가...?

for i in range(epochs):
    optimizer.zero_grad()
    # h_0 초기화
    hidden = rnn.init_hidden() #h(t=0)

    # 문자열 전체에 대한 loss
    total_loss = 0
    for j in range(one_hot.size()[0]-1): # 마지막 input에 대해서는 output밖에 없음.
        # input: t 시점의 글자
        input_ = one_hot[j:j+1, :] # 1x35 -> cat함수 적용하기 위해서.
        # output: t+1 시점의 글자
        target = one_hot[j+1] # vector
        output, hidden = rnn.forward(input_, hidden) # 매 t에서 hidden을 뽑아내고, t+1에서 hidden을 다시 input으로 집어넣는다.
        target = target.view(1, -1) # 1 x 35
        target = torch.argmax(target, -1) # 1짜리 class indice로 바꿈.
        loss = loss_fnc(output, target)
        total_loss += loss
    total_loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(loss)

tensor(3.5897, grad_fn=<NllLossBackward>)
tensor(4.3310, grad_fn=<NllLossBackward>)
tensor(3.2535, grad_fn=<NllLossBackward>)
tensor(2.2310, grad_fn=<NllLossBackward>)
tensor(0.4962, grad_fn=<NllLossBackward>)
tensor(0.0998, grad_fn=<NllLossBackward>)
tensor(0.0709, grad_fn=<NllLossBackward>)
tensor(0.0233, grad_fn=<NllLossBackward>)
tensor(0.0140, grad_fn=<NllLossBackward>)
tensor(0.0110, grad_fn=<NllLossBackward>)
tensor(0.0096, grad_fn=<NllLossBackward>)
tensor(0.0084, grad_fn=<NllLossBackward>)
tensor(0.0075, grad_fn=<NllLossBackward>)
tensor(0.0068, grad_fn=<NllLossBackward>)
tensor(0.0063, grad_fn=<NllLossBackward>)
tensor(0.0058, grad_fn=<NllLossBackward>)
tensor(0.0054, grad_fn=<NllLossBackward>)
tensor(0.0050, grad_fn=<NllLossBackward>)
tensor(0.0046, grad_fn=<NllLossBackward>)
tensor(0.0043, grad_fn=<NllLossBackward>)
tensor(0.0041, grad_fn=<NllLossBackward>)
tensor(0.0038, grad_fn=<NllLossBackward>)
tensor(0.0036, grad_fn=<NllLossBackward>)
tensor(0.0034, grad_fn=<NllLossBac

In [0]:
### Inference

start = torch.zeros(1,n_letters)
start[:,-2] = 1

rnn.eval()

hidden = rnn.init_hidden()
input_ = start # [BOS] token
output_string = ""

for i in range(len(string)):
    output, hidden = rnn.forward(input_, hidden)

    # 결과값을 문자로 바꿔서 output_string에 붙여줍니다.
    output_string += onehot_to_word(output.data)
    # 또한 이번의 결과값이 다음의 입력값이 됩니다.
    input_ = output

print(output_string)

hptttttttttttttttttttttttttttttttttttttttttttttttttttttttttttaaaaaaa
